In [1]:
from web3 import Web3
import pandas as pd
import json
import requests
from collections import OrderedDict

#### API 세팅

In [2]:
etherscan_apikey = 'ETHERSCANKEY'
infura_endpoints = 'https://mainnet.infura.io/v3/INFURAKEY'

In [3]:
w3 = Web3(Web3.HTTPProvider(infura_endpoints))
w3.isConnected()

True

#### 트랜잭션 및 스마트 컨트랙트 정보 조회

In [164]:
tx_hash = '0xe27739c11c9b107887763b17938ceca720e9dbca151c159a149bc3ffb321a5cc'

In [165]:
tx = w3.eth.get_transaction(tx_hash)

In [166]:
tx['to']

'0x76194f083cCdE0A111330F0f67252751A45f8E52'

In [167]:
abi_endpoint = f"https://api.etherscan.io/api?module=contract&action=getabi&address={tx['to']}&apikey={etherscan_apikey}"
abi = json.loads(requests.get(abi_endpoint).text)

In [168]:
contract = w3.eth.contract(address=tx["to"], abi=abi["result"])

In [169]:
funcs = [i for i in contract.functions]

In [170]:
pd.DataFrame([i for i in contract.functions], columns=['ABI info.'])

,ABI info.
0,allowance
1,approve
2,balanceOf
3,decimals
4,decreaseAllowance
5,increaseAllowance
6,name
7,symbol
8,totalSupply
9,transfer


In [173]:
func_params['recipient']

'0xaDAa77321735238C2303881932E824Ae8da296AD'

In [175]:
name=contract.functions.name().call()
symbol=contract.functions.symbol().call()
dec = contract.functions.decimals().call(); dec_powered=10**dec
func_obj, func_params = contract.decode_function_input(tx["input"])
amount = func_params['amount']/dec_powered
# amount = func_params['_value']/dec_powered
print('Source: ', tx['from'])
print('Target: ', func_params['recipient'])
# print('Target: ', func_params['_to'])
print('Token Address: ', tx['to'])
print('Token Name: ', name)
print('Token Symbol: ', symbol)
print('Decimals: ', dec)
# print('Object: ', func_obj)
print(f'Amount: {amount} {symbol}')

Source:  0x4149A8A3d4Cf0dF1B01083AC39F7943cE8A429e6
Target:  0xaDAa77321735238C2303881932E824Ae8da296AD
Token Address:  0x76194f083cCdE0A111330F0f67252751A45f8E52
Token Name:  IPRToken
Token Symbol:  IPR
Decimals:  18
Amount: 50000.0 IPR


In [176]:
data = OrderedDict()
data['Source'] = tx['from']
data['Target'] = func_params['recipient']
data['Token Address'] = tx['to']
data['Token Name'] = name
data['Token Symbol'] = symbol
data['Decimals'] = dec
data['Amount'] = amount
pd.DataFrame(data.items(), columns=['key', 'value'])

,key,value
0,Source,0x4149A8A3d4Cf0dF1B01083AC39F7943cE8A429e6
1,Target,0xaDAa77321735238C2303881932E824Ae8da296AD
2,Token Address,0x76194f083cCdE0A111330F0f67252751A45f8E52
3,Token Name,IPRToken
4,Token Symbol,IPR
5,Decimals,18
6,Amount,50000.0


In [179]:
balance=contract.functions.balanceOf(tx['to']).call()/(10**dec)
supply=contract.functions.totalSupply().call()/(10**dec)
print(f'Tokens with the contract : {balance} {symbol}')
print(f'Total Supply: {supply} {symbol}')

Tokens with the contract : 132670.0 IPR
Total Supply: 3000000000.0 IPR
